In [1]:
'''
    MNIST with CNN by PyTorch
'''

'\n    MNIST with CNN by PyTorch\n'

In [2]:
# Load Packages
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
# init for weights initialization by Xavier
import torch.nn.init
import torch.nn as nn

In [3]:
# CPU or GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

/opt/conda/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
'''
    parameters:
        learning_rate: necessary for GD procedure
        training_epochs: total number of learning
        batch_size: size of mini batch
'''
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
# call MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [6]:
# load MNIST dataset
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
# convolution layer output calculation
    '''
        Desc:
            1: number of channel
            1: number of filter
            11: filter size
    '''
conv = nn.Conv2d(1, 1, 11, stride=4, padding=0)
conv

IndentationError: unexpected indent (<ipython-input-7-e2a7c5c662c9>, line 2)

In [ ]:
input = torch.Tensor(1, 1, 227, 227)
input

In [ ]:
conv(input).shape

In [8]:
# basic CNN layer archi practice
input = torch.Tensor(1, 1, 28, 28)

# channel = 1 / filter = 5 / filter size = 5 x 5
conv1 = nn.Conv2d(1, 5, 5)

# kernel size = 2 x 2
pool = nn.MaxPool2d(2)

In [9]:
out = conv1(input)
out2 = pool(out)
print(out.size())
print(out2.size())

torch.Size([1, 5, 24, 24])
torch.Size([1, 5, 12, 12])


# MNIST with CNN Version_1

In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torchvision

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

/opt/conda/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# hyper parameters
learning_rate = 0.001
nb_epochs = 15
batch_size = 100

In [4]:
# call MNIST data
root = 'MNIST_data/'
train_data = dsets.MNIST(
    root=root,
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = dsets.MNIST(
    root=root,
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [5]:
# load dataset
train_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

In [6]:
# network architecture
class CNN(nn.Module):
    # class = 10 => 0 ~ 9
    def __init__(self, nb_class=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # FCL => input = 3136 / output = 10 (0 ~ 9)
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    # input => layer1 => layer2 => FCL
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        # FCL => flatten result
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
# model
model = CNN().to(device)

In [8]:
# cost and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# total size of train dataset
total_batch = len(train_loader)

for epoch in range(nb_epochs):
    avg_cost = 0
    for X, Y in train_loader:
        # X / Y => already tensor type
        X = X.to(device)
        Y = Y.to(device)
        
        outputs = model(X)
        cost = criterion(outputs, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        # total cost / total batch => avg cost
        avg_cost += cost / total_batch
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('[Train Finished]')

[Epoch:    1] cost = 0.220143586


In [ ]:
# eval with test dataset
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

# MNIST with CNN Version_2

In [22]:
# network architecture
class CNN_V2(nn.Module):
    # class = 10 => 0 ~ 9
    def __init__(self, nb_class=10):
        super(CNN_V2, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc1 = nn.Linear(3 * 3 * 128, 625, bias=True)
        self.relu = nn.ReLU()
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    # input => layer1 => layer2 => FCL
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        # FCL => flatten result
        out = out.view(out.size(0), -1)
        print(out.size(0))
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [23]:
model = CNN_V2().to(device)
value = (torch.Tensor(1, 1, 28, 28)).to(device)
value
model(value)

1


tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward>)